In [70]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import spearmanr, pearsonr
import pandas as pd
import random
import numpy as np
data = pd.read_csv("../datasets/explicitsim353.csv").values
data2 = pd.read_csv("../datasets/SimLex-999.txt", delimiter="\t")[["word1","word2","SimLex999"]].values
data3 = pd.read_csv("../datasets/SimVerb-3500.txt", delimiter="\t", header=None, usecols=[0,1,3]).values
data = np.concatenate((data, data2, data3), axis=0)
test = pd.DataFrame.from_dict({
    "word_1": data[:, 0],
    "word_2": data[:, 1],
    "label": data[:, 2],
})
print(test.shape)
test.head()

(4852, 3)


,word_1,word_2,label
0,Arafat,Jackson,2.051282
1,Arafat,peace,2.575
2,Arafat,terror,3.125
3,FBI,fingerprint,3.210526
4,FBI,investigation,4.789474


In [71]:
test_samples = [InputExample(texts=[x[0], x[1]], label=x[2]) for x in test.values.tolist()]
test_samples[0].texts, test_samples[0].label

(['Arafat', 'Jackson'], 2.05128205128)

In [72]:
word_embedding_model = models.Transformer("bert-base-cased")
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_bert = SentenceTransformer(modules=[word_embedding_model, pooling_model])

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(model_bert)

2024-01-01 22:34:44 - Use pytorch device_name: cuda
2024-01-01 22:34:44 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2024-01-01 22:34:47 - Cosine-Similarity :	Pearson: 0.0596	Spearman: 0.1217
2024-01-01 22:34:47 - Manhattan-Distance:	Pearson: 0.1299	Spearman: 0.1496
2024-01-01 22:34:47 - Euclidean-Distance:	Pearson: 0.1050	Spearman: 0.1391
2024-01-01 22:34:47 - Dot-Product-Similarity:	Pearson: 0.0119	Spearman: 0.0462


0.14963856420732158

In [73]:
word_embedding_model2 = models.Transformer("google/electra-base-discriminator")
pooling_model2 = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_electra = SentenceTransformer(modules=[word_embedding_model2, pooling_model2])

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(model_electra)

2024-01-01 22:34:48 - Use pytorch device_name: cuda
2024-01-01 22:34:48 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2024-01-01 22:34:52 - Cosine-Similarity :	Pearson: 0.0475	Spearman: 0.0484
2024-01-01 22:34:52 - Manhattan-Distance:	Pearson: 0.0472	Spearman: 0.0459
2024-01-01 22:34:52 - Euclidean-Distance:	Pearson: 0.0476	Spearman: 0.0462
2024-01-01 22:34:52 - Dot-Product-Similarity:	Pearson: 0.0471	Spearman: 0.0463


0.048398721025649974

In [74]:
import torch 
word_embedding_model = models.Transformer("bert-base-cased")
word_embedding_model.auto_model = torch.load("../output/bert-base-cased/mean/12_to_13/model_2024_01_01_03_16.pkl")
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_bert = SentenceTransformer(modules=[word_embedding_model, pooling_model])

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(model_bert)

2024-01-01 22:34:52 - Use pytorch device_name: cuda
2024-01-01 22:34:52 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2024-01-01 22:34:56 - Cosine-Similarity :	Pearson: 0.2397	Spearman: 0.2292
2024-01-01 22:34:56 - Manhattan-Distance:	Pearson: 0.2515	Spearman: 0.2331
2024-01-01 22:34:56 - Euclidean-Distance:	Pearson: 0.2464	Spearman: 0.2292
2024-01-01 22:34:56 - Dot-Product-Similarity:	Pearson: 0.2290	Spearman: 0.2220


0.23312377630761782

In [75]:
word_embedding_model2 = models.Transformer("google/electra-base-discriminator")
word_embedding_model2.auto_model = torch.load("../output/google-electra-base-discriminator/mean/12_to_13/model_2024_01_01_03_16.pkl")
pooling_model2 = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_electra = SentenceTransformer(modules=[word_embedding_model2, pooling_model2])

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(model_electra)

2024-01-01 22:34:57 - Use pytorch device_name: cuda
2024-01-01 22:34:57 - EmbeddingSimilarityEvaluator: Evaluating the model on  dataset:
2024-01-01 22:35:00 - Cosine-Similarity :	Pearson: 0.0566	Spearman: 0.0545
2024-01-01 22:35:00 - Manhattan-Distance:	Pearson: 0.0530	Spearman: 0.0481
2024-01-01 22:35:00 - Euclidean-Distance:	Pearson: 0.0507	Spearman: 0.0459
2024-01-01 22:35:00 - Dot-Product-Similarity:	Pearson: 0.0643	Spearman: 0.0630


0.0630188068112915

In [65]:
model_name = "google/electra-base-discriminator"
num_epochs = 50

In [66]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datasets/explicitsim353.csv").values
data2 = pd.read_csv("../datasets/SimLex-999.txt", delimiter="\t")[["word1","word2","SimLex999"]].values
data3 = pd.read_csv("../datasets/SimVerb-3500.txt", delimiter="\t", header=None, usecols=[0,1,3]).values
data = np.concatenate((data, data2, data3), axis=0)

train, dev = train_test_split(data, test_size=0.3)
dev, test = train_test_split(dev, test_size=0.5)

train_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in train.tolist()]
dev_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in dev.tolist()]
test_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in test.tolist()]


#### Just some code to print debug information to stdout
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)


model_save_path = '../output/word_sim_'+model_name.replace("/", "-")
for seed in range(1):
    logging.info("##### Seed {} #####".format(seed))
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    curr_model_save_path = model_save_path + '/seed_'+str(seed)

    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)
    train_loss = losses.CosineSimilarityLoss(model=model)

    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='wordsim_dev')

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
    logging.info("Warmup-steps: {}".format(warmup_steps))

    # Train the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=evaluator,
        epochs=num_epochs,
        #evaluation_steps=1000,
        warmup_steps=warmup_steps,
        output_path=curr_model_save_path
    )

    model = SentenceTransformer(curr_model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='wordsim_test')
    test_evaluator(model, output_path=curr_model_save_path)

2024-01-01 22:25:29 - ##### Seed 0 #####
2024-01-01 22:25:30 - Use pytorch device_name: cuda
2024-01-01 22:25:30 - Warmup-steps: 535


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 22:25:38 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 0:
2024-01-01 22:25:38 - Cosine-Similarity :	Pearson: 0.1482	Spearman: 0.1315
2024-01-01 22:25:38 - Manhattan-Distance:	Pearson: 0.1325	Spearman: 0.1252
2024-01-01 22:25:38 - Euclidean-Distance:	Pearson: 0.1315	Spearman: 0.1239
2024-01-01 22:25:38 - Dot-Product-Similarity:	Pearson: 0.1380	Spearman: 0.1253
2024-01-01 22:25:38 - Save model to output/word_sim_google-electra-base-discriminator/seed_0


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

...

2024-01-01 22:32:36 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 47:
2024-01-01 22:32:36 - Cosine-Similarity :	Pearson: 0.6562	Spearman: 0.6447
2024-01-01 22:32:36 - Manhattan-Distance:	Pearson: 0.6295	Spearman: 0.6168
2024-01-01 22:32:36 - Euclidean-Distance:	Pearson: 0.6296	Spearman: 0.6171
2024-01-01 22:32:36 - Dot-Product-Similarity:	Pearson: 0.6642	Spearman: 0.6515


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 22:32:45 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 48:
2024-01-01 22:32:45 - Cosine-Similarity :	Pearson: 0.6557	Spearman: 0.6445
2024-01-01 22:32:45 - Manhattan-Distance:	Pearson: 0.6285	Spearman: 0.6160
2024-01-01 22:32:45 - Euclidean-Distance:	Pearson: 0.6286	Spearman: 0.6163
2024-01-01 22:32:45 - Dot-Product-Similarity:	Pearson: 0.6640	Spearman: 0.6510


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 22:32:53 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 49:
2024-01-01 22:32:54 - Cosine-Similarity :	Pearson: 0.6560	Spearman: 0.6445
2024-01-01 22:32:54 - Manhattan-Distance:	Pearson: 0.6286	Spearman: 0.6158
2024-01-01 22:32:54 - Euclidean-Distance:	Pearson: 0.6287	Spearman: 0.6164
2024-01-01 22:32:54 - Dot-Product-Similarity:	Pearson: 0.6643	Spearman: 0.6512
2024-01-01 22:32:54 - Load pretrained SentenceTransformer: output/word_sim_google-electra-base-discriminator/seed_0
2024-01-01 22:32:54 - Use pytorch device_name: cuda
2024-01-01 22:32:54 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_test dataset:
2024-01-01 22:32:55 - Cosine-Similarity :	Pearson: 0.6791	Spearman: 0.6688
2024-01-01 22:32:55 - Manhattan-Distance:	Pearson: 0.6669	Spearman: 0.6570
2024-01-01 22:32:55 - Euclidean-Distance:	Pearson: 0.6684	Spearman: 0.6576
2024-01-01 22:32:55 - Dot-Product-Similarity:	Pearson: 0.6693	Spearman: 0.6615


In [76]:
model_name = "bert-base-cased"
num_epochs = 50

data = pd.read_csv("../datasets/explicitsim353.csv").values
data2 = pd.read_csv("../datasets/SimLex-999.txt", delimiter="\t")[["word1","word2","SimLex999"]].values
data3 = pd.read_csv("../datasets/SimVerb-3500.txt", delimiter="\t", header=None, usecols=[0,1,3]).values
data = np.concatenate((data, data2, data3), axis=0)

train, dev = train_test_split(data, test_size=0.3)
dev, test = train_test_split(dev, test_size=0.5)

train_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in train.tolist()]
dev_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in dev.tolist()]
test_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in test.tolist()]


#### Just some code to print debug information to stdout
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)


model_save_path = '../output/word_sim_'+model_name.replace("/", "-")
for seed in range(1):
    logging.info("##### Seed {} #####".format(seed))
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    curr_model_save_path = model_save_path + '/seed_'+str(seed)

    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)
    train_loss = losses.CosineSimilarityLoss(model=model)

    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='wordsim_dev')

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
    logging.info("Warmup-steps: {}".format(warmup_steps))

    # Train the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=evaluator,
        epochs=num_epochs,
        #evaluation_steps=1000,
        warmup_steps=warmup_steps,
        output_path=curr_model_save_path
    )

    model = SentenceTransformer(curr_model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='wordsim_test')
    test_evaluator(model, output_path=curr_model_save_path)

2024-01-01 23:21:54 - ##### Seed 0 #####
2024-01-01 23:21:54 - Use pytorch device_name: cuda
2024-01-01 23:21:54 - Warmup-steps: 535


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 23:22:03 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 0:
2024-01-01 23:22:04 - Cosine-Similarity :	Pearson: 0.3476	Spearman: 0.3248
2024-01-01 23:22:04 - Manhattan-Distance:	Pearson: 0.3505	Spearman: 0.3273
2024-01-01 23:22:04 - Euclidean-Distance:	Pearson: 0.3413	Spearman: 0.3199
2024-01-01 23:22:04 - Dot-Product-Similarity:	Pearson: 0.2905	Spearman: 0.2656
2024-01-01 23:22:04 - Save model to ../output/word_sim_bert-base-cased/seed_0


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

...

2024-01-01 23:29:12 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 47:
2024-01-01 23:29:13 - Cosine-Similarity :	Pearson: 0.7466	Spearman: 0.7344
2024-01-01 23:29:13 - Manhattan-Distance:	Pearson: 0.7269	Spearman: 0.7096
2024-01-01 23:29:13 - Euclidean-Distance:	Pearson: 0.7269	Spearman: 0.7089
2024-01-01 23:29:13 - Dot-Product-Similarity:	Pearson: 0.7387	Spearman: 0.7299


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 23:29:21 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 48:
2024-01-01 23:29:22 - Cosine-Similarity :	Pearson: 0.7461	Spearman: 0.7337
2024-01-01 23:29:22 - Manhattan-Distance:	Pearson: 0.7264	Spearman: 0.7087
2024-01-01 23:29:22 - Euclidean-Distance:	Pearson: 0.7264	Spearman: 0.7081
2024-01-01 23:29:22 - Dot-Product-Similarity:	Pearson: 0.7386	Spearman: 0.7298


Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

2024-01-01 23:29:30 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_dev dataset after epoch 49:
2024-01-01 23:29:31 - Cosine-Similarity :	Pearson: 0.7463	Spearman: 0.7340
2024-01-01 23:29:31 - Manhattan-Distance:	Pearson: 0.7265	Spearman: 0.7089
2024-01-01 23:29:31 - Euclidean-Distance:	Pearson: 0.7265	Spearman: 0.7084
2024-01-01 23:29:31 - Dot-Product-Similarity:	Pearson: 0.7388	Spearman: 0.7299
2024-01-01 23:29:31 - Load pretrained SentenceTransformer: ../output/word_sim_bert-base-cased/seed_0
2024-01-01 23:29:31 - Use pytorch device_name: cuda
2024-01-01 23:29:31 - EmbeddingSimilarityEvaluator: Evaluating the model on wordsim_test dataset:
2024-01-01 23:29:32 - Cosine-Similarity :	Pearson: 0.7353	Spearman: 0.7322
2024-01-01 23:29:32 - Manhattan-Distance:	Pearson: 0.7291	Spearman: 0.7230
2024-01-01 23:29:32 - Euclidean-Distance:	Pearson: 0.7286	Spearman: 0.7225
2024-01-01 23:29:32 - Dot-Product-Similarity:	Pearson: 0.7212	Spearman: 0.7200


In [77]:
model_name = "google/electra-base-generator"
num_epochs = 50

data = pd.read_csv("../datasets/explicitsim353.csv").values
data2 = pd.read_csv("../datasets/SimLex-999.txt", delimiter="\t")[["word1","word2","SimLex999"]].values
data3 = pd.read_csv("../datasets/SimVerb-3500.txt", delimiter="\t", header=None, usecols=[0,1,3]).values
data = np.concatenate((data, data2, data3), axis=0)

train, dev = train_test_split(data, test_size=0.3)
dev, test = train_test_split(dev, test_size=0.5)

train_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in train.tolist()]
dev_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in dev.tolist()]
test_samples = [InputExample(texts=[x[0],x[1]], label=x[2]/10) for x in test.tolist()]


#### Just some code to print debug information to stdout
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO,
    handlers=[LoggingHandler()]
)


model_save_path = '../output/word_sim_'+model_name.replace("/", "-")
for seed in range(1):
    logging.info("##### Seed {} #####".format(seed))
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False
    )

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    curr_model_save_path = model_save_path + '/seed_'+str(seed)

    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)
    train_loss = losses.CosineSimilarityLoss(model=model)

    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='wordsim_dev')

    warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
    logging.info("Warmup-steps: {}".format(warmup_steps))

    # Train the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=evaluator,
        epochs=num_epochs,
        #evaluation_steps=1000,
        warmup_steps=warmup_steps,
        output_path=curr_model_save_path
    )

    model = SentenceTransformer(curr_model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='wordsim_test')
    test_evaluator(model, output_path=curr_model_save_path)

2024-01-02 01:37:44 - ##### Seed 0 #####
2024-01-02 01:37:45 - Use pytorch device_name: cuda


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
